# Binaryzacja

### Zadanie domowe - binaryzacja adaptacyjna w oknach z interpolacją.

Pokazana w ramach podstawowej części ćwiczenia binaryzacja adaptacyjna działa dobrze, ale jest dość złożona obliczeniowo (choć oczywiście należy mieć świadomość, że implementację można zoptymalizować i wyeliminować pewne powtarzające się obliczenia).
Zbliżone rozwiązanie można również realizować w nieco innym wariancie - w oknach.
Ogólna idea jest następująca: wejściowy obraz dzielimy na nienachodzące (rozłączne) okna - wygodnie jest założyć, że są one kwadratowe i o rozmiarze będącym potęgą liczby 2.
W każdym z okien obliczamy próg - niech to będzie średnia i stosujemy do binaryzacji lokalnej.
Jak nietrudno się domyślić efekt nie będzie dobry, gdyż na granicach okien wystąpią artefakty.
Aby je wyeliminować należy zastosować interpolację, co zostanie szczegółowo opisane poniżej.
Warto zaznaczyć, że podobny mechanizm interpolacji stosowany jest w poznanym wcześniej algorytmie CLAHE.
Zainteresowane osoby odsyłam do artykułu na [Wikipedii](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization) oraz do artykułu o metodzie CLAHE - Zuiderveld, Karel. “Contrast Limited Adaptive Histograph Equalization.” Graphic Gems IV. San Diego: Academic Press Professional, 1994. 474–485.



Na początek zaimplementujemy wariant metody bez interpolacji:
1. Wczytaj obraz _rice.png_.
2. W dwóch pętlach `for`, dla okien o ustalonym wymiarze $W$ (potęga 2), oblicz średnią:
- pętle powinny mieć krok $W$,
- wynik (tj. średnie) należy zapisać w pomocniczej tablicy,
- przydatny operator to `//` - dzielenie całkowitoliczbowe (*floor division*).

3. W kolejnych dwóch pętlach `for` (tym razem o kroku 1) przeprowadź binaryzację z wyznaczonymi progami.
   Tu oczywiście należy się sprytnie odwołać do wyników z tablicy pomocniczej.
   Wyświetl wyniki - czy jest on poprawny?
   Podpowiedź - błędy lepiej widać dla mniejszego rozmiaru okna (np. 16 x 16).

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/'

fileName = "rice.png"
if not os.path.exists(fileName):
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

4. Rozwiązaniem problemu artefaktów na obrazie jest zastosowanie interpolacji.
   Próg binaryzacji dla danego piksela wyliczany jest na podstawie progów z sąsiednich okien.
   ![Ilustracja koncepcji interpolacji](https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/clahe_tile_interpolation.png)

   Koncepcja została przedstawiona na powyższym rysunku.
   Możliwe są 3 przypadki:
   - piksel leży w rogach obrazu (kolor czerwony) - wtedy za próg przyjmuje się wartość średniej obliczonej dla danego okna,
   - piksel leży na krawędzi obrazu (kolor zielony) - wtedy za próg przyjmuje się wartość obliczoną na podstawie średnich z dwóch sąsiednich okien,
   - piksel leży w środku (kolor fioletowy) - wtedy próg jest obliczany na podstawie 4 sąsiednich okien.

   Uwaga. Proszę zwrócić uwagę, że sprawa jest dość złożona.
   Obraz dzielimy na okna (dla nich liczymy średnią) i następnie każde z okien "wirtualnie" na cztery sub-okna (linie przerywane).
   To ułatwia znalezienie środków okien (czarne kwadraty), które są wykorzystywane w interpolacji.

5. Implementujemy interpolację.
   Potrzebujemy do tego znać progi (jeden, dwa lub cztery), ale dla przejrzystości obliczeń lepiej zawsze przyjąć cztery oraz odległości od rozważnego piksela do środka sąsiednich okien (też w ogólnym przypadku 4):
   - całość sprowadza się do określania pozycji piksela,
   - na początek rozważmy przypadek czterech narożników (kolor czerwony na rysunku) - trzeba napisać `if`, który je wyznaczy,
   - warto sprawdzić, czy nie popełniliśmy błędu i np. tymczasowo do obrazu wynikowego w tym miejscu przypisać wartość 255. Efekt powinien być taki, że widoczne będą tylko narożniki.
   - drugi przypadek do brzegi (kolor zielony) - postępujemy podobnie jak przy narożnikach, przy czym osobno wydzielamy brzegi pionowe i poziome. Tu też warto sobie obrazek "pokolorować".
   - na koniec wyznaczamy piksele w środku.
   - analizując poprawność proszę zwrócić uwagę na to, żeby nie było przerw pomiędzy obszarami.
   - mając podział możemy dla każdego z obszarów wyliczyć cztery progi ($t11, t12, t21, t22$):
        - dla narożników wartość ta będzie identyczna i wynosi po prostu `t11 =t[jT][iT]`, gdzie `iT=i//W` oraz `jT=j//W`.
          Uwaga. Proszę używać indeksów tymczasowych $jT,iT$, gdyż będą potrzebne w dalszych obliczeniach.
        - dla brzegów pionowych występują dwie wartości: okno bieżące i sąsiednie.
          Wyznaczenie współrzędnej poziomej jest proste (jak dla narożników).
          Nad współrzędną pionową trzeba się chwilę zastanowić - aby nie rozważać wielu przypadków można od bieżącej współrzędnej odjąć połowę rozmiaru okna i dopiero później wykonać dzielenie przez rozmiar okna.
          W ten sposób otrzymujemy indeks okna o mniejszej współrzędnej.
          Indeks drugiego uzyskamy dodając 1.
          Proszę się zastanowić dlaczego to działa - najlepiej to sobie rozrysować.
        - dla brzegów poziomych należy postąpić analogicznie,
        - obliczenia dla obszaru wewnątrz powinny być już oczywiste.
   - kolejny krok to wyliczenie odległości pomiędzy rozważanym pikselem, a czterema środkami.
     Przykładowo dla osi X wygląda to następująco: `dX1 = i - W/2 - iT*W` oraz `dX2 = (iT+1)*W - i-W/2`.
     Dla osi Y analogicznie.
     Ponownie proszę się zastanowić dlaczego to jest poprawne - najlepiej to sobie narysować.
   - ostatni krok to interpolacja dwuliniowa.
     Wykonamy ją w trzech krokach:
     - interpolacja w osi X dla dwóch górnych okien - sprowadza się ona do średniej ważonej pomiędzy wartościami $t11$ i $t12$, przy czym wagi to odpowiednio $dX2/W$ i $dX1/W$.
       Ponownie na podstawie rysunku proszę to przemyśleć.
     - interpolacja w osi X dla dolnych okien jest analogiczna,
     - interpolacja w osi Y również jest analogiczna, z tym, że wejściem są dwa wyniki interpolacji w poziomie.

6. "Kropka nad i" to oczywiście binaryzacja z wyznaczonym poprzez interpolację progiem - proszę dobrać rozmiar okna.
7. Na koniec proszę porównać na wspólnym rysunku poznane metody binaryzacji:
- Otsu,
- lokalna na podstawie średniej,
- lokalna Sauvoli,
- lokalna w oknach bez interpolacji,
- lokalna w oknach z interpolacją.

Proszę pod porównaniem, w osobnej sekcji *markdown*, krótko skomentować uzyskane wyniki.

In [ ]:
def calculate_auto_threshold(image):
    histogram = cv2.calcHist([image], [0], None, [256], [0, 256])
    pi = histogram / sum(histogram)
    pi_cum_sum = np.cumsum(pi)
    intensity_values = range(256)

    k = np.dot(intensity_values, pi)

    while True:
        P0 = pi_cum_sum[int(k)]
        m0 = np.dot(intensity_values[:int(k) + 1], pi[:int(k) + 1]) / P0 if P0 > 0 else 0

        P1 = 1 - P0
        m1 = np.dot(intensity_values[int(k) + 1:], pi[int(k) + 1:]) / P1 if P1 > 0 else 0

        k_new = (m0 + m1) / 2

        if abs(k_new - k) < 1:
            break

        k = k_new

    return k


In [ ]:
def otsu_threshold(image):
    histogram = cv2.calcHist([image], [0], None, [256], [0, 256])
    pi = histogram / np.sum(histogram)
    pi_cum_sum = np


    mean_intensity = np.dot(np.arange(256), pi)

    P0 = 0
    m = 0
    max_sigma_b2 = 0
    optimal_k = 0
    sigma_b2_values = np.zeros(256)

    for k in range(256):
        P0 += pi[k]
        m += k * pi[k]

        if P0 > 0 and P0 < 1:
            P1 = 1 - P0
            m1 = (mean_intensity - m) / P1

            sigma_b2 = P0 * P1 * (m1 - m / P0) ** 2
            sigma_b2_values[k] = sigma_b2

            if sigma_b2 > max_sigma_b2:
                max_sigma_b2 = sigma_b2
                optimal_k = k

    return (image > optimal_k).astype(np.uint8)


In [ ]:
def local_threshold(image, window_size=15):
    X, Y = image.shape
    W = window_size

    binary_image = np.zeros_like(image)

    for j in range(W // 2, Y - W // 2):
        for i in range(W // 2, X - W // 2):
            local_window = image[j - W // 2:j + W // 2 + 1, i - W // 2:i + W // 2 + 1]
            local_mean = np.mean(local_window)

            binary_image[j, i] = image[j, i] > local_mean

    return binary_image

In [ ]:
def sauvola_threshold(image, window_size=15, k=0.15, R=128):
    X, Y = image.shape
    W = window_size

    binary_image = np.zeros_like(image)
    for j in range(W // 2, Y - W // 2):
        for i in range(W // 2, X - W // 2):
            local_window = image[j - W // 2:j + W // 2 + 1, i - W // 2:i + W // 2 + 1]
            local_mean = local_window.mean()
            local_std = local_window.std()

            threshold = local_mean * (1 - k * ((local_std / R) - 1))
            binary_image[j, i] = image[j, i] > threshold

    return binary_image

In [ ]:
def adaptive_threshold_without_interpolation(image, window_size):
    rows, cols = image.shape

    averages = np.zeros((rows // window_size, cols // window_size), dtype=np.float32)

    for i in range(0, rows, window_size):
        for j in range(0, cols, window_size):
            window = image[i:i + window_size, j:j + window_size]
            averages[i // window_size, j // window_size] = np.mean(window)

    binary_image = np.zeros_like(image, dtype=np.uint8)

    for i in range(rows):
        for j in range(cols):
            iT, jT = i // window_size, j // window_size
            threshold = averages[iT, jT]
            binary_image[i, j] = image[i, j] > threshold

    return binary_image


In [ ]:
def bilinear_interpolation_thresholding(image, window_size):

    rows, cols = image.shape

    averages = np.zeros((rows // window_size, cols // window_size), dtype=np.float32)

    for i in range(0, rows, window_size):
        for j in range(0, cols, window_size):
            window = image[i:i + window_size, j:j + window_size]
            averages[i // window_size, j // window_size] = np.mean(window)

    binary_image = np.zeros_like(image, dtype=np.uint8)

    for i in range(rows):
        for j in range(cols):
            iT, jT = i // window_size, j // window_size

            t11 = averages[jT, iT]
            t12 = averages[jT, min(iT + 1, averages.shape[1] - 1)]
            t21 = averages[min(jT + 1, averages.shape[0] - 1), iT]
            t22 = averages[min(jT + 1, averages.shape[0] - 1), min(iT + 1, averages.shape[1] - 1)]

            dX1 = i - (iT * window_size + window_size // 2)
            dX2 = (iT + 1) * window_size + window_size // 2 - i
            dY1 = j - (jT * window_size + window_size // 2)
            dY2 = (jT + 1) * window_size + window_size // 2 - j

            dX1, dX2 = dX1 / window_size, dX2 / window_size
            dY1, dY2 = dY1 / window_size, dY2 / window_size

            t_upper = t11 * dX2 + t12 * dX1
            t_lower = t21 * dX2 + t22 * dX1

            threshold = t_upper * dY2 + t_lower * dY1

            binary_image[i, j] = image[i, j] > threshold

    return binary_image


In [ ]:
def plot_interpolation_results(original_image, binary_image, window_size):

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image, cmap="gray")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.title(f"Binary Image (W={window_size})")
    plt.imshow(binary_image, cmap="gray")
    plt.axis("off")

    plt.tight_layout()
    plt.show()


In [ ]:
img = cv2.imread("rice.png", cv2.IMREAD_GRAYSCALE)

window_size = 16

binary_otsu = otsu_threshold(img)
binary_local_mean = local_threshold(img)
binary_local_sauvola = sauvola_threshold(img)
binary_no_interp = adaptive_threshold_without_interpolation(img, window_size)
binary_interp = bilinear_interpolation_thresholding(img, window_size)

plt.figure(figsize=(15, 10))

plt.subplot(2, 3, 1)
plt.title("Original Image")
plt.imshow(img, cmap="gray")
plt.axis("off")

plt.subplot(2, 3, 2)
plt.title("Otsu")
plt.imshow(binary_otsu, cmap="gray")
plt.axis("off")

plt.subplot(2, 3, 3)
plt.title("Local Mean")
plt.imshow(binary_local_mean, cmap="gray")
plt.axis("off")

plt.subplot(2, 3, 4)
plt.title("Local Sauvola")
plt.imshow(binary_local_sauvola, cmap="gray")
plt.axis("off")

plt.subplot(2, 3, 5)
plt.title("No Interpolation")
plt.imshow(binary_no_interp, cmap="gray")
plt.axis("off")

plt.subplot(2, 3, 6)
plt.title("With Interpolation")
plt.imshow(binary_interp, cmap="gray")
plt.axis("off")

plt.tight_layout()
plt.show()